This notebook allows collecting results across multiple pipeline runs. Concatenating across sessions and bin sizes. 
It takes CSVs, then determines the most recent one from the filename. 

There are two major sets of decoded epochs - Laps and Ripples/Replays
There are two sets of marginals for the decoded epochs - the "by epoch" and "by time bin" marginals.
The "by time bin" epochs are a larger granulation, with each epoch consisting of one or more time bin.


```python

# 2024-01-23 - 
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-fet11-01_12-58-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-03_12-3-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_17-46-44_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_19-28-0_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-58-3_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-25-50_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_16-40-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_22-24-40_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_16-53-46_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_17-29-30_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_14-26-15_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_1-22-43_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-07_16-40-19_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_21-16-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_15-55-31_time_bin_size_sweep_results.h5



# found_session_export_paths = [Path(v).resolve() for v in  ["C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# ]]
```

In [2]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import sys
from copy import deepcopy
from typing import List, Dict, Tuple, Optional, Union, Callable
import pandas as pd
import re
from pathlib import Path
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

# matplotlib.use('Qt5Agg')
%matplotlib inline
# %matplotlib auto

# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
import seaborn as sns

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import find_csv_files, find_HDF5_files, find_most_recent_files, process_csv_file
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms

debug_print: bool = False

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Definitions

In [3]:
## Load across session t_delta CSV, which contains the t_delta for each session:
t_delta_csv_path = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs\2024-01-18_GL_t_split_df.csv').resolve() # Apogee
# t_delta_csv_path = Path('/home/halechr/cloud/turbo/Data/Output/collected_outputs/2024-01-18_GL_t_split_df.csv').resolve() # GL

# collected_outputs_directory = '/home/halechr/FastData/collected_outputs/'
# collected_outputs_directory = r'C:\Users\pho\Desktop\collected_outputs'
collected_outputs_directory = r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs' # APOGEE
# collected_outputs_directory = '/home/halechr/cloud/turbo/Data/Output/collected_outputs' # GL

## Find the files:
csv_files = find_csv_files(collected_outputs_directory)
h5_files = find_HDF5_files(collected_outputs_directory)

csv_sessions = find_most_recent_files(found_session_export_paths=csv_files)
h5_sessions = find_most_recent_files(found_session_export_paths=h5_files)


## The CSV containing the session delta time:
t_delta_df = pd.read_csv(t_delta_csv_path, index_col=0) # Assuming that your CSV file has an index column
t_delta_dict = t_delta_df.to_dict(orient='index')
# t_delta_df

# csv_sessions
# Extract each of the separate files from the sessions:
    
final_sessions = {}
final_sessions_loaded_laps_dict = {}
final_sessions_loaded_ripple_dict = {}
final_sessions_loaded_laps_time_bin_dict = {}
final_sessions_loaded_ripple_time_bin_dict = {}

for session_str, session_dict in csv_sessions.items():
    # try:
        final_sessions[session_str] = {}
        for file_type, (an_decoding_time_bin_size_str, a_path, an_export_datetime) in session_dict.items():
            final_sessions[session_str][file_type] = a_path
            
        session_name: str = str(session_str)  # Extract session name from the filename
        if debug_print:
            print(f'processing session_name: {session_name}')
        curr_session_t_delta: Optional[float] = t_delta_dict.get(session_name, {}).get('t_delta', None)
        if curr_session_t_delta is None:
            print(f'WARN: curr_session_t_split is None for session_str = "{session_str}"')

        # finds each of the four exports:
        try:
            laps_file = final_sessions[session_str]['laps_marginals_df']
            final_sessions_loaded_laps_dict[session_str] = process_csv_file(laps_file, session_name, curr_session_t_delta, 'lap_start_t')
        except BaseException as e:
            print(f'session {session_str} did not fully work. (error {e}. Skipping.') 
            pass   
                    
        try:
            ripple_file = final_sessions[session_str]['ripple_marginals_df']
            final_sessions_loaded_ripple_dict[session_str] = process_csv_file(ripple_file, session_name, curr_session_t_delta, 'ripple_start_t')
        except BaseException as e:
            print(f'session {session_str} did not fully work. (error {e}. Skipping.') 
            pass   
            

        try:
            laps_time_bin_file = final_sessions[session_str]['laps_time_bin_marginals_df']
            final_sessions_loaded_laps_time_bin_dict[session_str] = process_csv_file(laps_time_bin_file, session_name, curr_session_t_delta, 't_bin_center')
        except BaseException as e:
            print(f'session {session_str} did not fully work. (error {e}. Skipping.') 
            pass   


        try:
            ripple_time_bin_file = final_sessions[session_str]['ripple_time_bin_marginals_df']
            final_sessions_loaded_ripple_time_bin_dict[session_str] = process_csv_file(ripple_time_bin_file, session_name, curr_session_t_delta, 't_bin_center')
        except BaseException as e:
            print(f'session {session_str} did not fully work. (error {e}. Skipping.') 
            pass   


## Build across_sessions join dataframes:
all_sessions_laps_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_laps_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_ripple_dict.values()), axis='index', ignore_index=True)

# *_time_bin marginals:
all_sessions_laps_time_bin_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_laps_time_bin_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_time_bin_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_ripple_time_bin_dict.values()), axis='index', ignore_index=True)

# Filter rows based on column: 'time_bin_size'
all_sessions_laps_df = all_sessions_laps_df[all_sessions_laps_df['time_bin_size'].notna()]
all_sessions_ripple_df = all_sessions_ripple_df[all_sessions_ripple_df['time_bin_size'].notna()]
all_sessions_laps_time_bin_df = all_sessions_laps_time_bin_df[all_sessions_laps_time_bin_df['time_bin_size'].notna()]
all_sessions_ripple_time_bin_df = all_sessions_ripple_time_bin_df[all_sessions_ripple_time_bin_df['time_bin_size'].notna()]

all_sessions_laps_time_bin_df

ERR: Could not parse filename: "2024-01-18_GL_t_split_df"


,Unnamed: 0,P_LR,P_RL,P_Long,P_Short,epoch_idx,t_bin_center,session_name,delta_aligned_start_t,time_bin_size
19018,0,0.491114,0.508886,3.876855e-01,0.612315,0,6.907083,kdiba_vvp01_one_2006-4-10_12-25-50,-876.990049,0.03
19019,1,0.491114,0.508886,3.876855e-01,0.612315,0,6.937083,kdiba_vvp01_one_2006-4-10_12-25-50,-876.960049,0.03
19020,2,0.491114,0.508886,3.876855e-01,0.612315,0,6.967083,kdiba_vvp01_one_2006-4-10_12-25-50,-876.930049,0.03
19021,3,0.491114,0.508886,3.876855e-01,0.612315,0,6.997083,kdiba_vvp01_one_2006-4-10_12-25-50,-876.900049,0.03
19022,4,0.491114,0.508886,3.876855e-01,0.612315,0,7.027083,kdiba_vvp01_one_2006-4-10_12-25-50,-876.870049,0.03
...,...,...,...,...,...,...,...,...,...,...
627816,57452,0.993650,0.006350,8.458879e-01,0.154112,79,2068.387436,kdiba_gor01_one_2006-6-08_14-26-15,856.829356,0.10
627817,57453,0.875051,0.124949,6.168876e-07,0.999999,79,2068.487436,kdiba_gor01_one_2006-6-08_14-26-15,856.929356,0.10
627818,57454,0.703433,0.296567,2.924253e-01,0.707575,79,2068.587436,kdiba_gor01_one_2006-6-08_14-26-15,857.029356,0.10
627819,57455,0.928085,0.071915,8.841508e-01,0.115849,79,2068.687436,kdiba_gor01_one_2006-6-08_14-26-15,857.129356,0.10


In [4]:
# final_sessions
# {'kdiba_gor01_one_2006-6-08_14-26-15': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-17_0540PM-kdiba_gor01_one_2006-6-08_14-26-15-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-17_0540PM-kdiba_gor01_one_2006-6-08_14-26-15-(laps_marginals_df).csv')},
#  'kdiba_gor01_one_2006-6-09_1-22-43': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0838PM-kdiba_gor01_one_2006-6-09_1-22-43-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0838PM-kdiba_gor01_one_2006-6-09_1-22-43-(laps_marginals_df).csv')},
#  'kdiba_pin01_one_fet11-01_12-58-54': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv')}}


# Save out the four dataframes to CSVs:
from attrs import define, field, Factory

@define(slots=False)
class AcrossSessionCSVOutputFormat:
	data_description = ["AcrossSession"]
	epoch_description = ["Laps", "Ripple"]
	granularity_description = ["per-Epoch", "per-TimeBin"]
	
	parts_names = ["export_date", "date_name", "epochs", "granularity"]
	
	def parse_filename(self, a_filename: str):
		if a_filename.endswith('.csv'):
			a_filename = a_filename.removesuffix('.csv') # drop the .csv suffix
		# split on the underscore into the parts
		parts = a_filename.split('_')
		if len(parts) == 4:
			export_date, date_name, epochs, granularity  = parts
		else:
			raise NotImplementedError(f"a_csv_filename: '{a_filename}' expected four parts but got {len(parts)} parts.\n\tparts: {parts}")
		return export_date, date_name, epochs, granularity
	


final_dfs_dict = {"AcrossSession_Laps_per-Epoch": all_sessions_laps_df, "AcrossSession_Ripple_per-Epoch": all_sessions_ripple_df,
				   "AcrossSession_Laps_per-TimeBin": all_sessions_laps_time_bin_df, "AcrossSession_Ripple_per-TimeBin": all_sessions_ripple_time_bin_df}

final_output_path = Path("../output/").resolve()
TODAY_DAY_DATE: str = f"2024-01-29"

final_csv_export_paths = {}
for a_name, a_final_df in final_dfs_dict.items():
	# save out one final DF to csv.
	out_csv_filename: str = f"{TODAY_DAY_DATE}_{a_name}.csv"
	a_final_csv_export_path = final_output_path.joinpath(out_csv_filename).resolve()
	a_final_df.to_csv(a_final_csv_export_path) # save to CSV.
	final_csv_export_paths[a_name] = a_final_csv_export_path
	

final_csv_export_paths


{'AcrossSession_Laps_per-Epoch': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Ripple_per-TimeBin.csv')}

# Plotting functions

In [ ]:

# Example usage:
all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_df, concatenated_ripple_df=all_sessions_ripple_df, save_figures=True)

# Show figures for all sessions
for fig_laps, fig_ripples in all_session_figures:
    fig_laps.show()
    fig_ripples.show()

    fig_laps.write_html("../output/2024-01-23_AcrossSession_fig_laps.html")+
    fig_ripples.write_html("../output/2024-01-23_AcrossSession_fig_ripples.html")


In [ ]:
active_plot_fn = plot_across_sessions_scatter_results
# active_plot_fn = plot_across_sessions_results_with_histogram_gpt3
# active_plot_fn = plot_across_sessions_results_with_histogram_new
all_time_bin_session_figures = active_plot_fn(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df, save_figures=False)
all_time_bin_session_figures

for fig_time_bin_laps, fig_time_bin_ripples in all_time_bin_session_figures:
    fig_time_bin_laps.show()
    fig_time_bin_ripples.show()
    # fig_laps.write_html("../output/2024-01-18_AcrossSession_fig_laps.html")
    # fig_ripples.write_html("../output/2024-01-18_AcrossSession_fig_ripples.html")
    fig_time_bin_laps.write_html("../output/2024-01-23_AcrossSession_fig_time_bin_laps.html")
    fig_time_bin_ripples.write_html("../output/2024-01-23_AcrossSession_fig_time_bin_ripples.html")


In [ ]:
all_time_bin_session_figures[0].show()

In [ ]:

# from Spike3D.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:
plot_histograms('Laps', 'All Sessions', all_sessions_laps_time_bin_df, "75 ms")
plot_histograms('Ripples', 'All Sessions', all_sessions_ripple_time_bin_df, "75 ms")

In [ ]:
# You can use it like this:
num_unique_sessions: int = all_sessions_laps_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_laps_time_bin_df.time_bin_size.nunique(dropna=True)
plot_stacked_histograms('Laps', f'{num_unique_sessions} Sessions', all_sessions_laps_time_bin_df, f"{num_unique_time_bins} tbin sizes")

num_unique_sessions: int = all_sessions_ripple_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_ripple_time_bin_df.time_bin_size.nunique(dropna=True)
plot_stacked_histograms('Ripples', f'{num_unique_sessions} Sessions', all_sessions_ripple_time_bin_df, f"{num_unique_time_bins} tbin sizes")

In [ ]:
all_sessions_laps_time_bin_df.hist(column='P_Long')
plt.show()

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import plot_all_epoch_bins_marginal_predictions
from attrs import define, field, Factory
from neuropy.utils.result_context import IdentifyingContext

@define(slots=False)
class TwoCSV:
	""" simple class wrapper to emulate the object that holds the other dfs """
	laps_all_epoch_bins_marginals_df = field()
	ripple_all_epoch_bins_marginals_df = field()

save_figure = True
def _perform_write_to_file_callback(final_context, fig):
	print(f'final_context: {final_context}')
	# if save_figure:
	# 	fig.save_fig(
	# 	# return owning_pipeline_reference.output_figure(final_context, fig)
	# else:
	# 	pass # do nothing, don't save
	
# all_sessions_laps_time_bin_df

collector = plot_all_epoch_bins_marginal_predictions(TwoCSV(laps_all_epoch_bins_marginals_df=all_sessions_laps_df, ripple_all_epoch_bins_marginals_df=all_sessions_ripple_df), t_start=None, t_split=0.0, t_end=None,
													active_context=IdentifyingContext(), perform_write_to_file_callback=_perform_write_to_file_callback)
collector



In [ ]:
result = tuple(collector.figures)


In [ ]:
result[0].show()

# HDF5 files

In [ ]:
from neuropy.utils.result_context import IdentifyingContext
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import H5FileAggregator

h5_sessions = find_most_recent_files(found_session_export_paths=h5_files)
h5_sessions

final_sessions = {}
time_bin_size_sweep_results_files = {}

# final_sessions_loaded_laps_dict = {}
# final_sessions_loaded_ripple_dict = {}
# final_sessions_loaded_laps_time_bin_dict = {}
# final_sessions_loaded_ripple_time_bin_dict = {}

for session_str, session_dict in h5_sessions.items():
    final_sessions[session_str] = {}
    for file_type, (a_path, an_export_datetime) in session_dict.items():
        final_sessions[session_str][file_type] = a_path
        
    session_name: str = str(session_str)  # Extract session name from the filename
    if debug_print:
        print(f'processing session_name: {session_name}')
    curr_session_t_delta: Optional[float] = t_delta_dict.get(session_name, {}).get('t_delta', None)
    if curr_session_t_delta is None:
        print(f'WARN: curr_session_t_split is None for session_str = "{session_str}"')

    # finds each of the four exports:
    time_bin_size_sweep_results_file = final_sessions[session_str]['time_bin_size_sweep_results']
    time_bin_size_sweep_results_files[session_str] = Path(time_bin_size_sweep_results_file).resolve()
    # ripple_file = final_sessions[session_str]['ripple_marginals_df']
    # laps_time_bin_file = final_sessions[session_str]['laps_time_bin_marginals_df']
    # ripple_time_bin_file = final_sessions[session_str]['ripple_time_bin_marginals_df']
    


session_identifiers = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]

# Keys are like: "/kdiba|gor01|two|2006-6-12_16-53-46"
included_h5_paths = list(time_bin_size_sweep_results_files.values())
session_group_keys: List[str] = [("/" + a_ctxt.get_description(separator="|", include_property_names=False)) for a_ctxt in session_identifiers] # 'kdiba/gor01/one/2006-6-08_14-26-15'
laps_decoding_accuracy_results_table_keys = [f"{session_group_key}/laps_decoding_accuracy_results/table" for session_group_key in session_group_keys]
a_loader = H5FileAggregator.init_from_file_lists(file_list=included_h5_paths, table_key_list=laps_decoding_accuracy_results_table_keys)
_out_table = a_loader.load_and_consolidate()
_out_table